In [ ]:
from utils.db import load_all_rows_from_mongo
from utils.functions import processar_indicadores_financeiros
import pandas as pd
from prophet import Prophet
from statsmodels.tsa.stattools import adfuller
from statsmodels.stats.diagnostic import acorr_ljungbox
from statsmodels.tsa.holtwinters import SimpleExpSmoothing, ExponentialSmoothing
from sklearn.metrics import mean_absolute_error, mean_squared_error



c:\Users\guilh\AppData\Local\Programs\Python\Python39\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
all_rows = load_all_rows_from_mongo(limit=None)
df_hist = pd.DataFrame(all_rows)
indicadores_historicos = processar_indicadores_financeiros(df_hist)

In [13]:
indicadores_historicos["Lucro_Líquido"]

mes
2022-01    191244.34
2022-02    129539.87
2022-03    108017.63
2022-04     -1673.49
2022-05    107411.43
2022-06    140610.78
2022-07    201765.16
2022-08     42481.79
2022-09    -10005.80
2022-10    107998.54
2022-11    -51426.80
2022-12    161556.97
2023-01     97727.63
2023-02    -58321.78
2023-03    159333.27
2023-04     19981.44
2023-05    182849.04
2023-06     46135.77
2023-07     16101.75
2023-08   -180036.85
2023-09    111747.60
2023-10      1296.21
2023-11    167098.02
2023-12    -27760.40
2024-01    151418.81
2024-02    -70761.63
2024-03    137548.95
2024-04     99566.35
2024-05    142961.27
2024-06    103701.21
2024-07     55544.12
2024-08    268192.05
2024-09     99275.10
2024-10    174113.94
2024-11      -564.18
2024-12    133184.02
Name: Lucro_Líquido, dtype: float64

In [ ]:
resultado = adfuller(indicadores_historicos['Lucro_Líquido'])
p_valor = resultado[1]

print(f"p-valor: {p_valor:.4f}")

if p_valor < 0.05:
    print("✅ A série é estacionária (rejeita H₀).")
else:
    print("⚠️ A série NÃO é estacionária (não rejeita H₀). O Prophet consegue lidar com isso.")

p-valor: 0.0054
✅ A série é estacionária (rejeita H₀).


In [ ]:
lb = acorr_ljungbox(indicadores_historicos['Lucro_Líquido'], lags=[
                    12], return_df=True)
p_valor = lb['lb_pvalue'].iloc[-1]  # pega o p-valor do último lag

print(lb)
print(f"\nP-valor (lag 12): {p_valor:.4f}")

if p_valor < 0.05:
    print("✅ A série apresenta autocorrelação significativa (rejeita H₀).")
else:
    print("⚠️ A série NÃO apresenta autocorrelação significativa (não rejeita H₀).")

      lb_stat  lb_pvalue
12  17.376991   0.135958

P-valor (lag 12): 0.1360
⚠️ A série NÃO apresenta autocorrelação significativa (não rejeita H₀).


In [14]:
m = Prophet(yearly_seasonality=True,
            monthly_seasonality=False, daily_seasonality=False)
m.fit(indicadores_historicos["Lucro_Líquido"])

# Gerar previsões para 12 meses à frente
future = m.make_future_dataframe(periods=12, freq='M')
forecast = m.predict(future)

# Visualizar
m.plot(forecast)
m.plot_components(forecast)

TypeError: __init__() got an unexpected keyword argument 'monthly_seasonality'